mlflowserver: https://dagshub.com/yeyintaung.ya276/FaceRecognization/experiments

In [48]:
!git clone https://github.com/yeaung276/FaceRecognization.git
!pip install mlflow
%cd FaceRecognization

Cloning into 'FaceRecognization'...
remote: Enumerating objects: 6659, done.
remote: Counting objects: 100% (445/445), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 6659 (delta 193), reused 437 (delta 185), pack-reused 6214
Receiving objects: 100% (6659/6659), 371.99 MiB | 24.69 MiB/s, done.
Resolving deltas: 100% (349/349), done.
Updating files: 100% (403/403), done.


In [22]:
import tensorflow as tf
import mlflow
import time
import os

In [39]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'yeyintaung.ya276'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '32874b32af827497261af34e60fe79bbbe838adf'
mlflow.set_tracking_uri('https://dagshub.com/yeyintaung.ya276/FaceRecognization.mlflow')
def get_experiment_id(name):
  exp = mlflow.get_experiment_by_name(name)
  if exp is None:
    exp_id = mlflow.create_experiment(name)
    return exp_id
  return exp.experiment_id

In [40]:
feature_description = {
    'anchor': tf.io.FixedLenFeature([1024], tf.float32),
    'positive': tf.io.FixedLenFeature([1024], tf.float32),
    'negative': tf.io.FixedLenFeature([1024], tf.float32),
}
def parse_example(example_proto):
  result = tf.io.parse_single_example(example_proto, feature_description)
  return (result['positive'], result['anchor'], result['negative'])

In [41]:
training_data = tf.data.TFRecordDataset(
    './experiments/data/train-embeddings.gz',
    compression_type='GZIP').map(parse_example)
eval_data = tf.data.TFRecordDataset(
    './experiments/data/eval-embeddings.gz',
    compression_type='GZIP').map(parse_example)

In [42]:
class DistanceLayer(tf.keras.layers.Layer):
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, positive, anchor, negative):
      ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
      an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
      return ap_distance , an_distance

In [43]:
class TripletLoss(tf.keras.losses.Loss):
    def __init__(self, alpha, name="triplet_loss", **kwargs):
        super().__init__(name=name, **kwargs)
        self.margin = alpha

    def call(self, ap_distance, an_distance):
        loss = tf.reduce_mean(tf.maximum(ap_distance - an_distance + self.margin, 0.0))
        return loss

    def get_config(self):
        config = {
            'margin': self.margin
        }
        base_config = super().get_config()
        return {**base_config, **config}

def calc_accuracy(ap_d, an_d):
  count = ap_d.shape[0]
  return tf.reduce_sum(tf.cast(ap_d < an_d, dtype=tf.int16))/count

In [44]:
def create_model(
    intermediate_layer=32,
    output_layer=16,
    dropout_strength=0.4
):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(intermediate_layer,
              activation='relu'),
    tf.keras.layers.Dropout(dropout_strength),
    tf.keras.layers.Dense(output_layer,
              activation="relu")
  ])

  # Input Layers for the encoding
  anchor_input   = tf.keras.layers.Input(1024, name="anchor")
  positive_input = tf.keras.layers.Input(1024, name="positive")
  negative_input = tf.keras.layers.Input(1024, name="negative")

  ## Generate the encodings (feature vectors) for the images
  encoded_a = model(anchor_input)
  encoded_p = model(positive_input)
  encoded_n = model(negative_input)

  ## Calculate distance between anchor and positive/negative
  distances = DistanceLayer()(encoded_p, encoded_a, encoded_n)

  # Creating the Model
  siamese_model = tf.keras.models.Model(
          inputs  = [positive_input, anchor_input, negative_input],
          outputs = distances,
          name = "Siamese_Network"
      )
  return siamese_model, model

In [45]:
# hyperparameter
EXPERIMENT_NAME='checking_dataset'
INTERMEDIATE_LAYER=32
OUTPUT_LAYER=16
DROPOUT_STRENGTH=0.4
NO_EPOCH = 50
BATCH_SIZE = 128
lr = 1e-3
alpha=1.0

In [47]:
experiment_id = get_experiment_id(EXPERIMENT_NAME)
print(experiment_id)

with mlflow.start_run(experiment_id=experiment_id):
  siamese_model, model = create_model(
      intermediate_layer=INTERMEDIATE_LAYER,
      output_layer=OUTPUT_LAYER,
      dropout_strength=DROPOUT_STRENGTH
  )
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=1e-01)
  loss_eval = TripletLoss(alpha=alpha)
  mlflow.log_params({
     'intermediate_layer': INTERMEDIATE_LAYER,
     'output_layer': OUTPUT_LAYER,
     'dropout_strength': DROPOUT_STRENGTH,
     'epochs': NO_EPOCH,
     'batch_size': BATCH_SIZE,
     'learning_rate': lr,
     'margin_alpha': alpha
  })

  template = 'ETA: {} - epoch: {} loss: {:.5f} \
  acc: {:.3f}  val loss: {:.5f} val_acc: {:.3f}\n'

  train_loss_metric = tf.keras.metrics.Mean(name="loss")
  train_acc_metric = tf.keras.metrics.Mean(name="acc")
  train_an_distance = tf.keras.metrics.Mean(name="an_d")
  train_ap_distance = tf.keras.metrics.Mean(name="ap_d")

  test_loss_metric = tf.keras.metrics.Mean(name="loss")
  test_acc_metric = tf.keras.metrics.Mean(name="acc")
  test_an_distance = tf.keras.metrics.Mean(name="an_d")
  test_ap_distance = tf.keras.metrics.Mean(name="ap_d")

  metrics = []

  # start training loop
  for ep in range(1,NO_EPOCH+1):
    t = time.time()
    # training batch
    for n,a,p in training_data.batch(BATCH_SIZE):
      with tf.GradientTape() as tape:
        ap_d, an_d = siamese_model([n,a,p])
        train_loss = loss_eval(ap_d, an_d)
      grads = tape.gradient(train_loss, siamese_model.trainable_weights)
      optimizer.apply_gradients(
          zip(grads, siamese_model.trainable_weights)
      )
      train_loss_metric.update_state(train_loss)
      train_ap_distance.update_state(ap_d)
      train_an_distance.update_state(an_d)
      train_acc_metric.update_state(
          calc_accuracy(ap_d,an_d)
      )
    # testing batch
    for n,a,p in eval_data.batch(BATCH_SIZE):
      ap_d, an_d = siamese_model([n,a,p])
      test_loss = loss_eval(ap_d, an_d)
      test_loss_metric.update_state(test_loss)
      test_ap_distance.update_state(ap_d)
      test_an_distance.update_state(an_d)
      test_acc_metric.update_state(
          calc_accuracy(ap_d, an_d)
      )

    print(template.format(
        round(time.time() - t),
        ep,
        float(train_loss_metric.result()),
        float(train_acc_metric.result()),
        float(test_loss_metric.result()),
        float(test_acc_metric.result())
    ))
    metrics.append({
        "train_loss": train_loss_metric.result(),
        "test_loss": test_loss_metric.result(),
        "train_acc": train_acc_metric.result(),
        "test_acc": test_acc_metric.result(),
        "train_ap": train_ap_distance.result(),
        "test_ap": test_ap_distance.result(),
        "train_an": train_an_distance.result(),
        "test_an": test_an_distance.result()
    })
    mlflow.log_metrics({
        "train_loss": train_loss_metric.result(),
        "test_loss": test_loss_metric.result(),
        "train_acc": train_acc_metric.result(),
        "test_acc": test_acc_metric.result(),
        "train_ap": train_ap_distance.result(),
        "test_ap": test_ap_distance.result(),
        "train_an": train_an_distance.result(),
        "test_an": test_an_distance.result()
    }, step=ep)
    # clearing the metric values
    train_loss_metric.reset_states()
    train_acc_metric.reset_states()
    train_ap_distance.reset_states()
    train_an_distance.reset_states()
    test_loss_metric.reset_states()
    test_acc_metric.reset_states()
    test_ap_distance.reset_states()
    test_an_distance.reset_states()
  model.save('./model')
  mlflow.log_artifact('./model')


0
ETA: 6 - epoch: 1 loss: 1.38405   acc: 0.519  val loss: 0.99934 val_acc: 0.520

ETA: 4 - epoch: 2 loss: 0.98391   acc: 0.534  val loss: 0.97446 val_acc: 0.541

ETA: 6 - epoch: 3 loss: 0.95797   acc: 0.550  val loss: 0.95742 val_acc: 0.544

ETA: 6 - epoch: 4 loss: 0.92957   acc: 0.562  val loss: 0.95389 val_acc: 0.546

ETA: 6 - epoch: 5 loss: 0.90061   acc: 0.584  val loss: 0.92787 val_acc: 0.561

ETA: 6 - epoch: 6 loss: 0.87094   acc: 0.599  val loss: 0.91715 val_acc: 0.575

ETA: 4 - epoch: 7 loss: 0.84791   acc: 0.613  val loss: 0.90350 val_acc: 0.577

ETA: 4 - epoch: 8 loss: 0.82644   acc: 0.622  val loss: 0.89817 val_acc: 0.587

ETA: 4 - epoch: 9 loss: 0.80793   acc: 0.632  val loss: 0.89098 val_acc: 0.589

ETA: 6 - epoch: 10 loss: 0.79152   acc: 0.641  val loss: 0.89597 val_acc: 0.588

ETA: 11 - epoch: 11 loss: 0.78151   acc: 0.646  val loss: 0.88346 val_acc: 0.597

ETA: 6 - epoch: 12 loss: 0.76463   acc: 0.660  val loss: 0.89180 val_acc: 0.593

ETA: 5 - epoch: 13 loss: 0.75274  